## Data Cleaning 

The US Department of State produces the *Foreign Relations of the United States (FRUS)* in both print and online forms. Their online volumes are housed in a Github for public use in TEI XML <a href="https://github.com/HistoryAtState/frus">here</a>. While the markup in the files depict useful identifiers such as the officials involved, the type of document, et cetera, this project requires plain text format rather than encoded documents. Using an XQuery in Oxygen XML Editor, I removed the markup and introductory information, leaving only the contents within the <text> tag. This creates a unique version of the files particular to my project. The XQuery removes the <teiHeader> which contains metadata, the front material which includes the introductory publication statements, and table of contents, and the back information such as the index. The query, which can be found <a href="https://drive.google.com/file/d/1MkVhOUxD4IJbdr4Hg0z7JN83tt5Sc9Se/view?usp=sharing">here</a>, made each file into a plain text document.

## Word Vector Model Code

To begin, make sure that the you are in the correct location. Run the next chunk of code to ensure that the file path ends with "CloutierDHProject/Code_and_Data_Cleaning".
To "run" chunks of code, either press "command-control"/"command-return" or press "Run" above.

In [ ]:
getwd()

In [ ]:
install.packages("tidyverse")
install.packages("tidytext")
install.packages("magrittr")
install.packages("devtools")
install.packages("tsne")
install.packages("usethis")
install.packages("SnowballC")

In [ ]:
library(tidyverse)
library(tidytext)
library(magrittr)
library(devtools)
library(tsne)
library(usethis)
library(SnowballC)

In [ ]:
devtools::install_github('bmschmidt/wordVectors', force=TRUE)

In [ ]:
library(wordVectors)

For this next portion, please be sure to provide the correct file name. This path2file currently directs the code to the documents concerning Latin America during the Cold War. If you would rather analyze the European files, input "data/frusEU".

In [ ]:
path2file <- "data/frusLA"
fileList <- list.files(path2file,full.names = TRUE) 

readTextFiles <- function(file) { 
  message(file)
  rawText = paste(scan(file, sep="\n",what="raw",strip.white = TRUE))
  output = tibble(filename=gsub(path2file,"",file),text=rawText) %>% 
    group_by(filename) %>% 
    summarise(text = paste(rawText, collapse = " "))
  return(output)
}

combinedTexts <- tibble(filename=fileList) %>% 
  group_by(filename) %>% 
  do(readTextFiles(.$filename)) 

In the next chunk of code, users should rename the "file_name" to describe what model they are running.

In [ ]:
baseFile <- "file_name"
w2vInput <- paste("data/",baseFile,".txt", sep = "")
w2vCleaned <- paste("data/",baseFile,"_cleaned.txt", sep="")
w2vBin <- paste("data/",baseFile,".bin", sep="")
combinedTexts$text %>% write_lines(w2vInput)

## Parameters

Each model is based on the parameters that are selected prior to running the code. Word embedding models allow you to “choose how expansive you want the explored space to be” (Schmidt 2015). Tuning the parameters results in a greater accuracy depending on the analysis you are intending to complete. In order to test the usage of terms in the corpus, I tested a large variety of parameters in order to determine the accuracy of each model. For each set of parameters, both the Latin American and European corpora were tested and were run through six iterations on each corpus. Ultimately, there were six sets of parameters and therefore 62 models created in total.

In [ ]:
THREADS <- 3

prep_word2vec(origin=w2vInput,destination=w2vCleaned,lowercase=T,bundle_ngrams=1)

if (!file.exists(w2vBin)) {
  w2vModel <- train_word2vec(
    w2vCleaned,
    output_file=w2vBin,
    vectors=100,
    threads=THREADS,
    window=10, iter=10, negative_samples=5
  )
} else {
  w2vModel <- read.vectors(w2vBin)
}

If you choose to read in an existing .bin file, please see the next chunk of code. For example, I have included the .bin files for both Latin America documents and European documents that I use in the later analysis. If you would like to explore those documents instead of waitinig for a model to run, either input "LA6a.bin" for the corpus on Latin America, or "EU6a.bin" for the European-related corpus, replacing "file_name.bin". To use this code chunk, please remove the "#" before the code. "#" acts as a way to comment in the code, thus rendering the code inactive, if the user chooses. 

In [ ]:
#w2vModel <- read.vectors("data/LA6a.bin")

Here is an image of the model and some sample queries with terms of gender.

In [ ]:
w2vModel %>% plot(perplexity=10)

In [ ]:
w2vModel %>% closest_to("girl", 30)

In [ ]:
w2vModel %>% closest_to(~"girl"+"woman"+"girls"+"women", 20)

## Clusters

Clusters also present interesting representations of the model. As you can see by running the next chunk of code, clusters produce results that may be more predictable if you know the corpus well. For example, the corpus on Latin America in the Cold War produces clusters regarding regional resources, coastal fishiing, and more categories regarding hemispheric relations.
Please change "name_of_your_query" to a file name you would prefer.

In [ ]:
centers <- 150
clustering <- kmeans(w2vModel,centers=centers,iter.max = 40)

sapply(sample(1:centers, 10), function(n) {
  names(clustering$cluster[clustering$cluster==n][1:15])
})

w2vExport <-sapply(sample(1:centers,150),function(n) {
  names(clustering$cluster[clustering$cluster==n][1:15])
})

write.csv(file="output/name_of_your_query.csv", x=w2vExport)

## Validating the Model

The results of this project were extracted from the model iteration with the highest rate of similarity. The original code includes a short method for validating the model. I have included five more word pairs that are good representations of the corpus. These were instrumental in determining which iterations of the model were the strongest representations of the corpus. In the <a href="https://github.com/ccloutier312/CloutierDHProject/blob/main/Analysis.ipynb">analysis</a>, I use a single set of parameters on both corpora discussed. The files using these parameters are titled “LA6a” and "EU6a".

In [ ]:
files_list  = list.files(pattern="*.bin$", recursive=TRUE)

rownames <- c()

data_frame <- data.frame()
data = list(c("away", "off"),
            c("before", "after"),
            c("cause", "effects"),
            c("children", "parents"),
            c("come", "go"),
            c("day", "night"),
            c("first", "second"),
            c("good", "bad"),
            c("last", "first"),
            c("kind", "sort"),
            c("leave", "quit"),
            c("life", "death"),
            c("girl", "boy"),
            c("little", "small"),
            c("oil", "petroleum"),
            c("state", "department"),
            c("confidential", "secret"),
            c("east", "west"),
            c("aid", "assistance"))


data_list = list()

for(fn in files_list) {
  
  wwp_model = read.vectors(fn)
  sims <- c()
  for(pairs in data)
  {
    vector1 <- c()
    for(x in wwp_model[[pairs[1]]]) {
      vector1 <- c(vector1, x)
    }
    
    vector2 <- c()
    for(x in wwp_model[[pairs[2]]]) {
      vector2 <- c(vector2, x)
    }
    
    sims <- c(sims, cosine(vector1, vector2))
    f_name <- strsplit(fn, "/")[[1]][[2]]
    data_list[[f_name]] <- sims
  }
  
}

for(pairs in data) {
  rownames <- c(rownames, paste(pairs[1], pairs[2], sep="-"))
}

results <- structure(data_list,
                     class     = "data.frame",
                     row.names = rownames
)

write.csv(file="output/model-test-results.csv", x=results)